In [1]:
# turn this into app

In [2]:
%load_ext autoreload
%autoreload 2

import requests, bs4, re
import time, os
import psycopg2, json
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

from scraper import betfair, neds


In [3]:
STAKE = 50
COMM = 0.06

In [4]:
chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--window-size=1920x1080")

browser = webdriver.Chrome(executable_path='/Users/jake/Downloads/chromedriver', options=chrome_options)
browser.set_window_size(1920, 1080)
browser2 = webdriver.Chrome(executable_path='/Users/jake/Downloads/chromedriver', options=chrome_options)
browser2.set_window_size(1920, 1080)

In [5]:
nds = 'https://www.neds.com.au/racing/'
bf = 'https://www.betfair.com.au/exchange/plus/horse-racing/next'
browser.get(nds)
browser2.get(bf)

## Manually Navigate to desired race

In [6]:
def main(neds_df, df_bf):
    final_df = neds_df.join(df_bf, how='outer')
    final_df = final_df[['num', 'runner', 'back', 'lay', 'lay_vol', 'back_bf', 'back_vol', 'place', 'flucs']]
    for c in ['back', 'lay', 'back_bf', 'place']:
        final_df[c] = final_df[c].apply(float)

    final_df['EV_noback'] = (STAKE * final_df.back - STAKE) * 1/final_df.back_bf + (0.7 * STAKE - STAKE) * (1 / final_df.place - 1/final_df.back_bf) + -STAKE * (1 - 1/final_df.place)
    final_df['EV_half'] = (STAKE * final_df.back - STAKE - STAKE/2 * (final_df.lay-1)) * 1/final_df.back_bf + (0.7 * STAKE - STAKE + STAKE/2 * (1 - COMM)) * (1 / final_df.place - 1/final_df.back_bf) + (-STAKE + STAKE/2 * (1 - COMM)) * (1 - 1/final_df.place)

    return final_df[final_df.lay != 0].sort_values('EV_half', ascending=False)

In [7]:
button = widgets.Button(description="Run/Refresh")
button_cl = widgets.Button(description="Close")
display(button)
display(button_cl)
out = widgets.Output()

@out.capture()
def on_button_clicked(b):
    # final_df = neds_df.merge(df_bf, how='outer', on='num')
    neds_df = neds(browser)
    df_bf = betfair(browser2)
    df = main(neds_df, df_bf)
    
    clear_output()
    display(df)
    
def close(b):
    browser.close()
    browser2.close()

button.on_click(on_button_clicked)
button_cl.on_click(close)
out

Button(description='Run/Refresh', style=ButtonStyle())

Button(description='Close', style=ButtonStyle())

Output()

In [8]:
# silk = re.compile('\([\d]+\)')

# html_neds = browser.page_source
# df_list = pd.read_html(html_neds)
# if df_list[0].shape[1] == 11:
#     neds_df = df_list[0].iloc[:,:4].dropna().reset_index(drop=True)
#     neds_df.columns = ['runner', 'mi', 'flucs', 'odds']
# if df_list[0].shape[1] == 14:
#     neds_df = df_list[0].iloc[:,:4].dropna().reset_index(drop=True)
#     neds_df.columns = ['runner', 'best_time', 'flucs', 'odds']
# else:
#     neds_df = df_list[0].iloc[:,:3].dropna().reset_index(drop=True)
#     neds_df.columns = ['runner', 'flucs', 'odds']    

# neds_df['back'] = neds_df.odds.apply(lambda x: x.split(' ')[0])
# neds_df['place'] = neds_df.odds.apply(lambda x: x.split(' ')[1])
# neds_df.drop('odds', axis=1, inplace=True)

# neds_df['num'] = neds_df.runner.apply(lambda x: silk.findall(x)[0])

# neds_df